In [1]:
import pandas as pd
import otrl

In [2]:
hard_policy = pd.DataFrame(index=range(11, 18), columns=range(1, 11), dtype=int)
soft_policy = pd.DataFrame(index=range(17, 21), columns=range(1, 11), dtype=int)

print(hard_policy)
print(soft_policy)

    1   2   3   4   5   6   7   8   9   10
11 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
12 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
13 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
14 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
15 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
16 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
17 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
    1   2   3   4   5   6   7   8   9   10
17 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
18 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
19 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
20 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN


In [3]:
states_rl = pd.read_csv("../pretrained_models/blackjack/blackjack_states.csv")
outcomes_rl = pd.read_csv("../pretrained_models/blackjack/blackjack_outcomes.csv")
action_rl = (outcomes_rl.idxmax(axis=1) == outcomes_rl.columns[1]).astype(int)
pretrained_mdl = otrl.EnumeratedPolicy(
    [tuple(items) for items in states_rl.values.tolist()],
    action_rl,
    states_rl.columns,
    outcomes_rl.columns
)

In [7]:
policy_hard = pd.read_csv("../pretrained_models/blackjack/blackjack_optimal_hard.csv").to_dict()
policy_soft = pd.read_csv("../pretrained_models/blackjack/blackjack_optimal_soft.csv").to_dict()
policy_full = {}
for states, action in policy_hard:
    policy_full[states[0], states[1], 0] = action
print(policy_full)
# outcomes_true = pd.read_csv("../pretrained_models/blackjack/blackjack_outcomes.csv")
# action_true = (outcomes_rl.idxmax(axis=1) == outcomes_rl.columns[1]).astype(int)
# pretrained_mdl = otrl.EnumeratedPolicy(
#     [tuple(items) for items in states_rl.values.tolist()],
#     action_rl,
#     states_rl.columns,
#     outcomes_rl.columns
# )

ValueError: too many values to unpack (expected 2)

In [4]:
models = {
    "OCT": otrl.OTRLPretrained(f"trained/blackjack/json/oct.json"),
    "OPT": otrl.OTRLPretrained(f"trained/blackjack/json/opt.json"),
    "Pretrained": pretrained_mdl
}

hard_policies = {}
soft_policies = {}

for name, mdl in models.items():
    hard_policies[name] = hard_policy.copy()
    soft_policies[name] = soft_policy.copy()

In [6]:
for name, mdl in models.items():
    for player in hard_policies[name].index:
        for dealer in hard_policies[name].columns:
            action = mdl.predict((player, dealer, 0))[0]
            hard_policies[name].loc[player, dealer] = int(action)

for name, mdl in models.items():
    for player in soft_policies[name].index:
        for dealer in soft_policies[name].columns:
            action = mdl.predict((player, dealer, 1))[0]
            soft_policies[name].loc[player, dealer] = int(action)

print(hard_policies["Pretrained"])
print(soft_policies["Pretrained"])

     1    2    3    4    5    6    7    8    9    10
11  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
12  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
13  1.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0
14  1.0  1.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0
15  1.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0
16  1.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0
17  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
     1    2    3    4    5    6    7    8    9    10
17  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
18  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0
19  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
20  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
